In [26]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
import itertools
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [5]:
url='https://converge360.com/pages/summit-list/summits.aspx'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
items=soup.find('ul',{'class':'events-list'}).find_all('li')
len(items)

cards=[]
for q in items:
    ritle=' '.join(q.find('div',{'class':'seminarname'}).text.split())
    link=q.find('div',{'class':'seminarname'}).find('a')['href']
    info=' '.join(q.find('div',{'class':'blurb'}).text.split())
    da=' '.join(q.find('div',{'class':'blurb'}).text.split())
    cards.append([ritle,link,info,da])


In [47]:
converge=[]
for k in cards:
    link=k[1]
    driver.get(link)
    time.sleep(3)
    soupn=bs(driver.page_source,'lxml')
    
    title=' '.join(soupn.find('title').text.split()).split('-')[0].strip()
    dat=' '.join(soupn.find('p',{'class':'date-time'}).text.split())
    dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
    for f in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Mon','Tue','Wed','Thu','Fri','Sat','Sun']:
        if f in dat:
            dat=dat.replace(f,'').strip()
    
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat.strip()
    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.split('and')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.split('&')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()

    else:
        st=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
        en=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust').strip()
   
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        if not re.search('(\d{4})',en):
            en=en+' '+str(datetime.today().year)
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
    
    event_info=' '.join(soupn.find('div',{'id':'ph_pcontent3_0_divBody'}).text.split())
    ag=[' '.join(a.text.split()) for a in soupn.find('div',{'class':'agenda-schedule'}).find('div').findChildren() if ' '.join(a.text.split())!='']
    def mygrouper(n, iterable):
        args = [iter(iterable)] * n
        return ([e for e in t if e != None] for t in itertools.zip_longest(*args))
    agen=list(mygrouper(2, ag))

    agenda=[]
    for idx,age in enumerate(agen):
        timiti=age[0]
        timi=timiti.split(',',1)[0].strip().split('/')[0].replace('. ','{').replace('.','').replace(' ',':00').upper().split('{')[0]
        ry=timi
        try:
            qt=agen[idx+1][0].split(',',1)[0].strip().split('/')[0].replace('. ','{').replace('.','').replace(' ',':00').upper().split('{')[0]
        except:
            qt=''
        ttl=timiti.split(',',1)[1].replace(':','').strip()
        desc=age[1]
        arrai={
                'start_time':ry,
                'end_time':qt,
                'day':start_date,
                'desc':desc,
                'title':ttl

                }
        agenda.append(arrai)
    agendalist=json.dumps(agenda)#,ensure_ascii=False)
    xx=agen[0][0].split(',',1)[0].strip().split('/')[0].replace('. ','{').replace('.','').replace(' ',':00').upper().split('{')[0]
    yy=agen[-1][0].split(',',1)[0].strip().split('/')[0].replace('. ','{').replace('.','').replace(' ',':00').upper().split('{')[0]
    z=agen[-1][0].split(',',1)[0].strip().split('/')[0].replace('. ','{').replace('.','').replace(' ',':00').upper().split('{')[1]
    time_st={
            'type':'general',
            'Start_time':xx,
            'end_time':yy,
            'timezone':z,
            'days':'all'
        }
    time_=json.dumps([time_st],ensure_ascii=False)
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    org_name="1105 Media's Converge360 group"
    org_web='https://converge360.com/'
    org_pro="1105 Media's Converge360 group connects you to the high-level, experienced, enterprise tech professional you want to reach – C-Level, Directors & IT Managers, Architects, Security Analytics, Development Managers, Data Professionals, and more. We have launched more than 60 tech media properties, hundreds of web sites and over 1,000+ newsletters. We have 14 million pages of published content and counting from our expert tech writers, editors and designers. We have launched more than 50 print magazines and have served over 350,000+ live event attendees."
    logo='' 
    sp=soupn.find('section',{'class':'right'}).find_all('span')
    spon=[' '.join(a.find('img')['src']).split('/')[-1].split('.')[0].replace(' ','').replace('White','') for a in sp]
    sponsor=json.dumps(spon,ensure_ascii=False)
    Speakerlist=''
    type_=''
    category=''
   
    ticket_list=''
    event_web=link
    #mail_=json.dumps(['projectmanagers@accessmba.com'])
    mail_=''
    
    loc=' '.join(soupn.find('div',{'class':'why-attend'}).text.split())
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc or 'ebinar' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        
        on_off='0'
        city=loc.split(',')[0].strip()
        country=loc.split(',')[-1].strip()
        if ven=='':
            venue=loc
        else:
            venue=ven
        googlePlaceUrl=get_google_map_url(venue)
        
    
    converge.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])

In [48]:
len(converge)

19

In [49]:
converge

[['https://redmondmag.com/Microsites/2022/Securing-Managing-and-Recovering-Microsoft-365/Home.aspx',
  'Securing, Managing & Recovering Microsoft 365 Summit',
  '2022-07-08',
  '2022-07-08',
  '[{"type": "general", "Start_time": "9:00AM", "end_time": "11:00AM", "timezone": "PDT", "days": "all"}]',
  "Microsoft 365 is key in most enterprises today, but all aspects of the software are constantly evolving, as is what you need to do as an IT manager to secure, optimize and recover these tools. You can't afford security breaks, downtime, latency issues or bad practices. That's why you won't want to miss this free half-day summit from the editors of Redmondmag.com that will keep you ahead of those changes and give you real-world insights you can use the same day to improve your Microsoft 365 installation and make sure it's secure and recoverable in the year ahead. Don't miss it!",
  '',
  "1105 Media's Converge360 group connects you to the high-level, experienced, enterprise tech professiona

In [50]:
converge_dict=converge


converge_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=converge_dict)
converge_df.to_csv("converge.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [ ]:
# unox=[]
# for k in range(1,31):
#     driver.switch_to.window(driver.window_handles[k])

#     soupn=bs(driver.page_source,'lxml')
#     title=' '.join(soupn.find('header',{'class':'main-title'}).text.split())
#     try:
#         dat=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-date.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         dat=''
    
#     dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').strip()
#     if '(' in dat:
#         dat=dat.split('(')[0].strip()
#     if dat=='':
#         start_date=end_date=''
#     else:
#         mad=dat
#     if '-' in mad:
#         st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#     elif 'and'in mad:
#         st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#     elif '&' in mad:
#         st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#     else:
#         st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
   
#     if any(c.isalpha() for c in st)==False:
#     #      print('leg')
#         pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
#         sapa=pa.group()
#         start=st+' '+sapa
#         end=en
#     #                        02        April      2022
#     elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
#      #   print('awujale')
#         start=st
#         end=en

#     #*                        April       02      2022
#     elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
#     #       print('ala')
#         pa=re.search('(\d{4})',st).group()
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()
#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba     

#     #                      02     2022
#     elif re.search('\d{1,2}\s+\d{4}',en):
#     #      print('kum')
#         pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',st).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba

#     #                      02         April
#     elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
#     #        print('is')
#         pa=re.search('(\d{4})',en)#, maxsplit=0)
#         sapa=pa.group()
#         start=st+' '+sapa
#         end=en

#     #*                      April           02
#     elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
#     #      print('bad')
#         pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba

#     else:
#     #     print('shik')
#         start=end=''
#     if start=='':
#         start_date=end_date=''
#     else:
#         pick=[start, end]
#         try:
#             spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
#         except:
#             spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
#         date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
#         start_date=date_[0]
#         end_date=date_[1]

   
#     try:
#         tim=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-time.+')}).find('span',{'class':'field-item-single'}).text.split()).upper().replace('TO','-').split('-')#.strip()
#     except:
#         tim=''
#     try:
#         tic=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-cost.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         tic=''
#     if tic!='':
#         if 'Free' in tic or tic=='Free':
#             ty='free'
#             am=''
#             cu=''

#         elif re.search('\$?\£?\€?\W?[0-9]+',tic).group():
#             ty='paid'
#             am=re.search('[0-9]+',tic).group()
#             cu=re.search('\$?\£?\€?',tic).group()
#         ticket=[{'type': ty,
#                  'price':am,
#                  'currency': cu}]
#         ticket_list=json.dumps(ticket,ensure_ascii=False)
#     else:
#         ticket_list=''
    
    
#     try:
#         event_info=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-body.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         event_info=''
#     try:
#         event_web=soupn.find('div',{'class':re.compile('field field-name-field-further-information.+')}).find('a')['href']
#     except:
#         event_web=''

        
#     if tim=='':
#         time_=''
        
#     else:
#         #print('azure')
#         if len(tim)==2:
#             stt=datetime.strptime(tim[0].strip(), '%H:%M').strftime('%I:%M%p')
#             ent=datetime.strptime(tim[1].strip(), '%H:%M').strftime('%I:%M%p')
#             z=''
#         elif len(tim)==1:
#             stt=datetime.strptime(tim[0].strip(), '%H:%M').strftime('%I:%M%p')
#             ent=''
#             z=''
#         time_st={   
#             'type':'general',
#             'Start_time':stt,
#             'end_time':ent,
#             'timezone':z,
#             'days':'all'
#         }

#         time_=json.dumps([time_st])
# #     #**************
#     def gu(luc):
#         google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
#         time.sleep(randint(0,3))
#         driver.get(google_url_for_location)
#         time.sleep(4)
#         try:
#             google_map_url=driver.find_element_by_id('lu_map').click()
#         except:
#             try:
#                 google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
#             except:
#                 google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
#         time.sleep(1)
#         google_map_url=driver.current_url
# #                 print(google_map_url)
#         return(google_map_url)
#         ######################################
#     def get_google_map_url(location):
#         try:
#             return(gu(location))
#         except:
#             try:
#                 return(gu(location+', University of Michigan'))
#             except:
#                 sha=location.split(',')
#                 try:
#                     gu(sha[-3])
#                 except:
#                     try:
#                         gu(sha[-2])
#                     except:
#                         try:
#                             gu(sha[-1])
#                         except Exception as e:
#                             print(location, "; url didn't go through")
#                             print(e)
#                             return("")


#     def countr(locale):
#         try:
#             google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
#             time.sleep(randint(0,3))
#             driver.get(google_url_for_location)
#             time.sleep(4)
#             try:
#                 google_map_url=driver.find_element_by_id('lu_map').click()
#             except:
#                 try:
#                     google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
#                 except:
#                     try:
#                         google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
#                     except:
#                         locale=locale+', University of Michigan'
#                         google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
#                         time.sleep(randint(0,3))
#                         driver.get(google_url_for_location)
#                         time.sleep(4)
#                         google_map_url=driver.find_element_by_id('lu_map').click()
#             time.sleep(1)
#             try:
#                 country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
#             except:
#                 country=driver.find_element_by_class_name('bwoZTb').text
# #                 print(country)
#             return(country)
#         except Exception as e:
#             print(locale, ";country didn't go through")
#             print(e)
#             return("")

#     try:
#         loc=' '.join(soupn.find('div',{'class':re.compile('field field-name-field-event-venue.+')}).find('span',{'class':'field-item-single'}).text.split())
#     except:
#         loc=''

#     org_name='University of Oxford'
#     org_web='https://www.ox.ac.uk/'
#     org_pro='Oxford has a distinctive collegiate structure. Students and academics benefit from belonging both to the University, a large, internationally-renowned institution, and to a college or hall, a small, interdisciplinary academic community.'
#     logo=''    
#     sponsor=''
#     agendalist=''
#     type_=''
#     category=''
#     Speakerlist='' 

#     try:
#         mail_=json.dumps[' '.join(soupn.find('div',{'class':re.compile('field field-name-field-contact-email.+')}).text.split())]
#     except:
#         mail_=''
#     if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
#         on_off='1'
#         city=''
#         venue=''
#         country=''
#         googlePlaceUrl=''
#     else:
#         on_off='0'
#         if ',' in loc:
#             city=loc.split(',')[-2].strip()
#             country=loc.split(',')[-1].strip()
#         else:
#             city=loc.split()[-3]
#             country=countr(city)
#         venue=loc
#         googlePlaceUrl=get_google_map_url(venue)


#     unox.append([link,title,start_date,end_date,time_,event_info,ticket_list,
#                  org_pro,org_name,org_web,logo,sponsor,agendalist,
#                  type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
#                  Speakerlist,on_off])

# unox    
# #print('Done')

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
